# Nowcasting Test Cases

I previously archived HRRR output for CONUS during Hurricane Harvey. Here, we extract the now-casting relevant fields from each file and archive as an intermediate netCDF file. 

**Fields**

- U component of storm motion
- V component of storm motion
- CAPE
- Precipitation Rate
- (Reflectivity)

** Case locations**

1. 8/28 @ 03Z - 35.629N, 1{0,2}.327W
2. 8/28 @ 04Z - 35.057N, 102.722W

First we need to download the data from the Utah archive; the data we archived from `ldm` is missing precipitation rate. There has been a mis-communication on where William is scraping is HRRR fields, or otherwise or forecast precipitation data in HyperCast is *very* wrong.

In [1]:
from __future__ import print_function

from collections import namedtuple
import os

fcst = namedtuple('forecast', ['month', 'day', 'hour', 'fcst_hour'])

forecasts_a = [fcst(8, 28, 2, i) for i in range(7)]
forecasts_b = [fcst(8, 28, 3, i) for i in range(7)]

ROOT = (
    "https://pando-rgw01.chpc.utah.edu/HRRR/oper/sfc/"
    "2017{fcst.month:02d}{fcst.day:02d}/hrrr.t{fcst.hour:02d}z.wrfsfcf{fcst.fcst_hour:02d}.grib2"
)
NEW_FN = "hrrr.2017{fcst.month:02d}{fcst.day:02d}.t{fcst.hour:02d}z.f{fcst.fcst_hour:02d}.grib2"
f = forecasts_a[0]
print(ROOT.format(fcst=f))
print(NEW_FN.format(fcst=f))

https://pando-rgw01.chpc.utah.edu/HRRR/oper/sfc/20170828/hrrr.t02z.wrfsfcf00.grib2
hrrr.20170828.t02z.f00.grib2


Download Case "A"

In [2]:
if not os.path.exists('case_a'):
    os.makedirs("case_a")
for f in forecasts_a:
    fn = ROOT.format(fcst=f)
    new_fn = NEW_FN.format(fcst=f)
    !wget -O case_a/{new_fn} {fn} 

--2017-09-12 20:29:16--  https://pando-rgw01.chpc.utah.edu/HRRR/oper/sfc/20170828/hrrr.t02z.wrfsfcf00.grib2
Resolving pando-rgw01.chpc.utah.edu... 155.101.11.35
Connecting to pando-rgw01.chpc.utah.edu|155.101.11.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103964180 (99M) [application/octet-stream]
Saving to: ‘case_a/hrrr.20170828.t02z.f00.grib2’

case_a/hrrr.2017082 100%[===================>]  99.15M  9.95MB/s    in 10s     

2017-09-12 20:29:28 (9.47 MB/s) - ‘case_a/hrrr.20170828.t02z.f00.grib2’ saved [103964180/103964180]

--2017-09-12 20:29:28--  https://pando-rgw01.chpc.utah.edu/HRRR/oper/sfc/20170828/hrrr.t02z.wrfsfcf01.grib2
Resolving pando-rgw01.chpc.utah.edu... 155.101.11.35
Connecting to pando-rgw01.chpc.utah.edu|155.101.11.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107935955 (103M) [application/octet-stream]
Saving to: ‘case_a/hrrr.20170828.t02z.f01.grib2’

case_a/hrrr.2017082 100%[===================>] 102.94

Download Case "B"

In [3]:
if not os.path.exists('case_b'):
    os.makedirs("case_b")
for f in forecasts_b:
    fn = ROOT.format(fcst=f)
    new_fn = NEW_FN.format(fcst=f)
    !wget -O case_b/{new_fn} {fn} 

--2017-09-12 20:31:22--  https://pando-rgw01.chpc.utah.edu/HRRR/oper/sfc/20170828/hrrr.t03z.wrfsfcf00.grib2
Resolving pando-rgw01.chpc.utah.edu... 155.101.11.35
Connecting to pando-rgw01.chpc.utah.edu|155.101.11.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104138636 (99M) [application/octet-stream]
Saving to: ‘case_b/hrrr.20170828.t03z.f00.grib2’

case_b/hrrr.2017082 100%[===================>]  99.31M  10.5MB/s    in 13s     

2017-09-12 20:31:36 (7.60 MB/s) - ‘case_b/hrrr.20170828.t03z.f00.grib2’ saved [104138636/104138636]

--2017-09-12 20:31:36--  https://pando-rgw01.chpc.utah.edu/HRRR/oper/sfc/20170828/hrrr.t03z.wrfsfcf01.grib2
Resolving pando-rgw01.chpc.utah.edu... 155.101.11.35
Connecting to pando-rgw01.chpc.utah.edu|155.101.11.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108234501 (103M) [application/octet-stream]
Saving to: ‘case_b/hrrr.20170828.t03z.f01.grib2’

case_b/hrrr.2017082 100%[===================>] 103.22

We now want to create a subset of the data. We'll read in each forecast dataset, read it in via **xarray**, and select a 2.5 x 2.5 degree box centered on the locations indicated for both of the test cases. We can subset the data to just include the fields used by the now-castin engine. Once we have this sub-selection complete, we'll unravel our tensor data into a vector which we can record as a CSV file.

Each case will have 7 forecast CSV files - I've taken the model run initialized *one hour before* the observation time, so that we can capture a 7-hour forecast progression.

In [6]:
import pandas as pd
import xarray as xr

fields_map = [
    ('USTM_P0_2L103_GLC0', 'u_stm'),
    ('VSTM_P0_2L103_GLC0', 'v_stm'),
    ('CAPE_P0_L1_GLC0', 'cape'),
    ('PRATE_P0_L1_GLC0', 'precip_rate'),
    ('REFD_P0_L103_GLC0', 'refl'),
    ('gridlat_0', 'lat'),
    ('gridlon_0', 'lon')
]
old_fields, new_fields = map(list, zip(*fields_map))

OUT_FN = "hrrr_subset.2017{fcst.month:02d}{fcst.day:02d}.t{fcst.hour:02d}z.f{fcst.fcst_hour:02d}.csv"


Case A -  8/28 @ 03Z - 35.629N, 1{0,2??}.327W

In [7]:
for f in forecasts_a[:1]:
    fn = 'case_a/' + NEW_FN.format(fcst=f)
    print("\n", f, fn)
    out_fn = 'case_a/' + OUT_FN.format(fcst=f)

    # Process dataset
    ds = xr.open_dataset(fn, engine='pynio')[old_fields]
    ds = (
        ds
        .rename({old: new for old, new in fields_map})
        .assign(time=[pd.Timestamp("2017-{fcst.month:02d}-{fcst.day:02d} {hour:02d}:00:00"
                                   .format(fcst=f, hour=f.hour + f.fcst_hour)), ])
        .set_coords('time')
    )
    ds = (
        ds
        .where((32.5 < ds.lat) & (ds.lat < 37.5) & 
               (-104.5 < ds.lon) & (ds.lon < -99.5), drop=True)
        .sel(lv_HTGL8=1000.)
        .squeeze()
    )
    ds.to_netcdf(out_fn.replace("csv", "nc"))
    
    # Convert to DataFrame
    df = (
        ds
        .to_dataframe()
        .dropna()
        .reset_index(drop=True)
        [new_fields]
    )
    
    print("writing", out_fn)
    df.to_csv(out_fn, index=False)



 forecast(month=8, day=28, hour=2, fcst_hour=0) case_a/hrrr.20170828.t02z.f00.grib2
writing case_a/hrrr_subset.20170828.t02z.f00.csv


Case B - 8/28 @ 04Z - 35.057N, 102.722W

In [8]:
for f in forecasts_b[:1]:
    fn = 'case_b/' + NEW_FN.format(fcst=f)
    print("\n", f, fn)
    
    out_fn = 'case_b/' + OUT_FN.format(fcst=f)

    # Process dataset
    ds = xr.open_dataset(fn, engine='pynio')[old_fields]
    ds = (
        ds
        .rename({old: new for old, new in fields_map})
        .assign(time=[pd.Timestamp("2017-{fcst.month:02d}-{fcst.day:02d} {hour:02d}:00:00"
                                   .format(fcst=f, hour=f.hour + f.fcst_hour)), ])
        .set_coords('time')
    )
    ds = (
        ds
        .where((32.5 < ds.lat) & (ds.lat < 37.5) & 
              (-105 < ds.lon) & (ds.lon < -100), drop=True)
        .sel(lv_HTGL8=1000.)
        .squeeze()
    )
    ds.to_netcdf(out_fn.replace("csv", "nc"))
    
    # Convert to DataFrame
    df = (
        ds
        .to_dataframe()
        .dropna()
        .reset_index(drop=True)
        [new_fields]
    )
    
    print("writing", out_fn)
    df.to_csv(out_fn, index=False)



 forecast(month=8, day=28, hour=3, fcst_hour=0) case_b/hrrr.20170828.t03z.f00.grib2
writing case_b/hrrr_subset.20170828.t03z.f00.csv
